## Step 1: Import Libraries

In [ ]:

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, concatenate
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns


## Step 2: Load Asteroseismology Data

In [ ]:

df = pd.read_csv("seismo_data.csv")  # Load dataset

print("First 5 rows of the dataset:")
display(df.head())

# Extract oscillation and stellar features
X_osc = df.iloc[:, :-6].values.reshape(len(df), 200, 1)
X_star_params = df.iloc[:, -6:-1].values
y_labels = df['label'].values

# Train-test split
X_osc_train, X_osc_test, X_star_train, X_star_test, y_train, y_test = train_test_split(
    X_osc, X_star_params, y_labels, test_size=0.2, random_state=42
)


## Step 3: Build the Asteroseismic AI Model

In [ ]:

def build_seismic_model(input_shape_osc, input_shape_star):
    """Hybrid CNN + Dense Model for Exoplanet Detection"""
    
    # Asteroseismic CNN Branch (Oscillations Time-Series)
    input_osc = Input(shape=input_shape_osc, name="oscillation_input")
    x = Conv1D(filters=32, kernel_size=5, activation='relu')(input_osc)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(filters=64, kernel_size=5, activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)

    # Stellar Property Dense Branch
    input_star = Input(shape=input_shape_star, name="stellar_input")
    y = Dense(64, activation='relu')(input_star)
    y = Dropout(0.3)(y)
    y = Dense(32, activation='relu')(y)

    # Merge branches
    merged = concatenate([x, y])
    final_output = Dense(1, activation='sigmoid')(merged)

    model = Model(inputs=[input_osc, input_star], outputs=final_output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Build model
model = build_seismic_model(input_shape_osc=(200, 1), input_shape_star=(5,))


## Step 4: Train the Model

In [ ]:

history = model.fit(
    [X_osc_train, X_star_train], y_train,
    epochs=20, batch_size=32,
    validation_data=([X_osc_test, X_star_test], y_test)
)


## Step 5: Evaluate the Model

In [ ]:

test_loss, test_acc = model.evaluate([X_osc_test, X_star_test], y_test)
print(f"Test Accuracy: {test_acc:.4f}")


## Step 6: Visualize Training Performance

In [ ]:

plt.figure(figsize=(8, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy over Epochs')
plt.show()


## Step 7: Make Predictions

In [ ]:

new_oscillation = X_osc_test[0:1]  # Select sample
new_star_params = X_star_test[0:1]

# Predict probability
prediction = model.predict([new_oscillation, new_star_params])
print(f"Predicted Exoplanet Probability: {prediction[0][0]:.4f}")


## Step 8: Save the Model

In [ ]:

model.save("seismic_exoplanet_model.h5")  # Save model
print("Model Saved Successfully!")
